In [1]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

system("ls ../input")
diabetes <- read.csv("../input/diabetes.csv", header = TRUE)
head(diabetes)
# Any results you write to the current directory are saved as output.

In [2]:
# first look at the data set using summary() and str() to understand what type of data are you working
# with
summary(diabetes)
str(diabetes)
print("---")
print("Number of positive and negative Outcome samples:")
table(diabetes$Outcome)

In [3]:
# dropping null values in coloumns:
    # Glucose	BloodPressure	SkinThickness	Insulin	BMI	DiabetesPedigreeFunction

for (i in 2:6) {
      diabetes <- diabetes[-which(diabetes[, i] == 0), ]
}


In [4]:
positive_diabetes <- diabetes[diabetes$Outcome == 1, ]

negative_diabetes <- diabetes[diabetes$Outcome == 0, ]

In [5]:
# How varaibles are distributed

library(reshape2)

gg <- melt(diabetes)
ggplot(gg, aes(x=value, fill=variable)) +
  geom_histogram(binwidth=10)+
  facet_wrap(~variable)

In [6]:
# Finding the correlation between the attributs
library(ggcorrplot)

cc<-cor(diabetes)
corr<-round(cc,1)
ggcorrplot(corr, hc.order = FALSE, 
           type = "lower", 
           lab = TRUE, 
           lab_size = 3, 
           method="circle", 
           colors = c("red", "white", "blue"), 
           title="Correlogram of Diabetes data", 
           ggtheme=theme_bw)

In [7]:
# calcolo delle pdf senza dividere diab da non diab
# controllare se pressione è gaussiana
# dividere età per gruppi
# anova


# density plot
plot(density(positive_diabetes$BloodPressure), 
     xlim = c(0, 150),
     ylim = c(0.00, 0.04),
     xlab = "Glucose Level",
     main = "Density plot",
     lwd = 2)
curve(dnorm(x,mean(positive_diabetes$BloodPressure),sd(positive_diabetes$BloodPressure)),0,250,add=T,col="red")
legend("topleft", 
       col = c("black", "red"), 
       legend = c("Distribuzione BloodPressure", "Gaussian mean(BloodPressure) sd(BloodPressure)"), 
       lwd = 2,
       bty = "n")


# test di normalità su BloodPressure
shapiro.test(positive_diabetes$BloodPressure)
ks.test(positive_diabetes$BloodPressure, pnorm, mean(positive_diabetes$BloodPressure), sd(positive_diabetes$BloodPressure))

# divisione in classi
c1 <- positive_diabetes[positive_diabetes$Age <= 30, 3]
c2 <- positive_diabetes[positive_diabetes$Age > 30 & positive_diabetes$Age <= 50, 3]
c3 <- positive_diabetes[positive_diabetes$Age > 50, 3]


In [8]:
par(mfrow = c(2, 2))

plot(density(c1))
curve(dnorm(x,mean(c1),sd(c1)),0,250,add=T,col="red")
plot(density(c2))
curve(dnorm(x,mean(c2),sd(c2)),0,250,add=T,col="red")
plot(density(c3))
curve(dnorm(x,mean(c3),sd(c3)),0,250,add=T,col="red")


library(nortest)
# test di normalità su BloodPressure nelle classi
shapiro.test(c1)
ks.test(c1, pnorm, mean(c1), sd(c1))
lillie.test(c1)


shapiro.test(c2)
ks.test(c2, pnorm, mean(c2), sd(c2))
lillie.test(c2)

shapiro.test(c3)
ks.test(c3, pnorm, mean(c3), sd(c3))
lillie.test(c3)




In [9]:
qqnorm(c1)
qqline(c1)

qqnorm(c2)
qqline(c2)

qqnorm(c3)
qqline(c3)


In [10]:
# test omosessualità
positive_diabetes$gruppo <- positive_diabetes$Age

for (index in 1:nrow(positive_diabetes)){
    if(positive_diabetes[index,8]<=30){
        positive_diabetes[index,10]<-1
    }else if (positive_diabetes[index,8]>30 & positive_diabetes[index,8]<=50){
        positive_diabetes[index,10]<-2
    }else if (positive_diabetes[index,8]>50){
        positive_diabetes[index,10]<-3
    }
}

head(positive_diabetes)
bartlett.test(positive_diabetes$BloodPressure,positive_diabetes$gruppo)

aov(positive_diabetes$BloodPressure ~ positive_diabetes$gruppo)
resultava <- aov(positive_diabetes$BloodPressure ~ positive_diabetes$gruppo)
summary(resultava)

In [34]:
#boxplot(c1,c2,c3)

library(gplots)
plotmeans(positive_diabetes$BloodPressure ~ positive_diabetes$gruppo)

In [33]:
# blodpress per diab e non
par(mfrow = c (2,2))
qqnorm(positive_diabetes$BloodPressure)
qqline(positive_diabetes$BloodPressure)

qqnorm(negative_diabetes$BloodPressure)
qqline(negative_diabetes$BloodPressure)

shapiro.test(positive_diabetes$BloodPressure)
shapiro.test(negative_diabetes$BloodPressure)

bartlett.test(diabetes$BloodPressure,diabetes$Outcome)

aov(diabetes$BloodPressure ~ diabetes$Outcome)
resultava <- aov(diabetes$BloodPressure ~ diabetes$Outcome)
summary(resultava)

mpb<-mean(positive_diabetes$BloodPressure)
mnb<-mean(negative_diabetes$BloodPressure)
mpb
mnb

plotmeans(diabetes$BloodPressure ~ diabetes$Outcome)



In [ ]:
#create density plot for single variable

library(gridExtra)
library(grid)
library(class)
library(base)
library(caret)
library(gmodels)

diabetes$DiabetesResult <- factor(diabetes$Outcome, levels = c("0", "1"), labels = c("negative", "positive"))

dia_preg <- ggplot(diabetes, aes(x=Pregnancies, colour=factor(Outcome), fill=factor(Outcome))) +
    geom_density(stat = "density",alpha=.3) + 
    geom_vline(aes(xintercept=mean(Pregnancies)),linetype="dashed",color="black", size=1)+
    xlab("Pregnancies") +  
    ylab("Density")+
    theme_bw()+
    theme(legend.position="none",panel.border = element_blank(),
        axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_bmi <- ggplot(diabetes, aes(x=BMI, colour = factor(Outcome), fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept =mean(BMI)),linetype="dashed",color="black",size=1) + 
    xlab("BMI values") +  
    ylab("Density")+
    theme_bw()+
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_skin <- ggplot(diabetes, aes(x=SkinThickness, colour = factor(Outcome), fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(SkinThickness)),linetype="dashed",color="black",size=1) +
    xlab("Skin Thickness") +  
    ylab("Density") +
    theme_bw() +
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_age <- ggplot(diabetes, aes(x=Age, colour = factor(Outcome), fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(Age)),linetype="dashed",color="black",size=1) +
    xlab("Age") +  
    ylab("Density") +
    theme_bw() +
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1")  

dia_ins <- ggplot(diabetes, aes(x=Insulin, colour = factor(Outcome), fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(Insulin)),linetype="dashed",color="black",size=1) +
    xlab("Insulin") +  
    ylab("Density") +
    theme_bw() +
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_pedf <- ggplot(diabetes, aes(x=DiabetesPedigreeFunction, colour = factor(Outcome), 
                                 fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(DiabetesPedigreeFunction)),linetype="dashed",
           color="black",size=1) +
    xlab("Diabetes Pedigree Function") +  
    ylab("Density") +
    theme_bw() +
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_bpres <- ggplot(diabetes, aes(x=BloodPressure, colour = factor(Outcome), 
                                  fill = factor(Outcome))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(BloodPressure)),linetype="dashed",color="black",size=1) +
    xlab("BloodPressure") +  
    ylab("Density") +
    theme_bw() +
    theme(legend.position="none",panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1") +
    scale_fill_brewer(palette = "Set1") 

dia_glu <- ggplot(diabetes, aes(x=Glucose, colour = factor(DiabetesResult), 
                                fill = factor(DiabetesResult))) + 
    geom_density(alpha=0.3) +
    geom_vline(aes(xintercept = mean(Glucose)),linetype="dashed",color="black",size=1) +
    xlab("Glucose") +  
    ylab("Density") +
    theme_bw()+ 
    #scale_colour_discrete(name ="Outcome",labels=c("0", "1"))+
    #scale_fill_discrete(name  ="Outcome",labels=c("0", "1"))+
    theme(legend.position="bottom",legend.direction="horizontal",legend.title=element_text(size=8), 
      legend.text=element_text(size=7),panel.border = element_blank(),
      axis.line = element_line(colour = "black"))+
    scale_colour_brewer(palette = "Set1",name = "Diabetes") +
    scale_fill_brewer(palette = "Set1",name = "Diabetes")



grid.arrange(dia_preg + ggtitle(""),
             dia_bmi  + ggtitle(""),
             dia_skin + ggtitle(""),
             dia_age + ggtitle(""),
             dia_bpres + ggtitle(""),
             dia_ins + ggtitle(""),
             dia_pedf +ggtitle(""),
             dia_glu + ggtitle(""),
             
             nrow = 3,
             top = textGrob("Diabetes outcome density plot for selected variables", 
                            gp=gpar(fontsize=14))
)

#-----------------------------------------------
#mettere gluc all'inizio

In [ ]:
# polt hist: glucose,BMI
par(mfrow = c(2, 2))

# the $ notation can be used to subset the variable you're interested in.

#GLUCOSIO
hist(positive_diabetes$Glucose,xlim=c(0,250),ylim=c(-0.0005,0.015),prob=T)
lines(density(positive_diabetes$Glucose),col="blue")
curve(dnorm(x,mean(positive_diabetes$Glucose),sd(positive_diabetes$Glucose)),0,250,add=T,col="red")

#ETA
hist(positive_diabetes$Age,xlim=c(0,100),ylim=c(-0.0005,0.07),prob=T)
lines(density(positive_diabetes$Age),col="blue")
curve(dnorm(x,mean(positive_diabetes$Age),sd(positive_diabetes$Age)),0,250,add=T,col="red")

#GRAVIDANZE
hist(positive_diabetes$Pregnancies,xlim=c(-10,30),ylim=c(-0.0005,0.21),prob=T)
lines(density(positive_diabetes$Pregnancies),col="blue")
curve(dnorm(x,mean(positive_diabetes$Pregnancies),sd(positive_diabetes$Pregnancies)),0,250,add=T,col="red")

#BMI
hist(positive_diabetes$BMI,xlim=c(0,100),ylim=c(-0.0005,0.12),prob=T)
lines(density(positive_diabetes$BMI),col="blue")
curve(dnorm(x,mean(positive_diabetes$BMI),sd(positive_diabetes$BMI)),0,250,add=T,col="red")

#INSULINA
hist(positive_diabetes$Insulin,xlim=c(0,800),ylim=c(-0.0005,0.01),prob=T)
lines(density(positive_diabetes$Insulin),col="blue")
curve(dnorm(x,mean(positive_diabetes$Insulin),sd(positive_diabetes$Insulin)),0,250,add=T,col="red")

#--------------------------------
# cavare i meno correlati, lasci insulina

In [ ]:
par(mfrow = c(2,2))

boxplot_glucose_outcome<- ggplot(diabetes,aes(x=DiabetesResult,y=Glucose,
                        color=factor(DiabetesResult))) + 
geom_boxplot(alpha=0.7)+
theme_bw() +
scale_colour_brewer(palette = "Set1",name = "Diabetes")
boxplot_glucose_outcome

boxplot_age_outcome<- ggplot(diabetes,aes(x=DiabetesResult,y=Age,
                        color=factor(DiabetesResult))) + 
geom_boxplot(alpha=0.7)+
theme_bw() +
scale_colour_brewer(palette = "Set1",name = "Diabetes")
boxplot_age_outcome


In [ ]:
# print boxplot value

positive <- subset(diabetes,diabetes$Outcome == 1)
print("POSITIVI: ")
print("_glucosio")
summary(positive$Glucose)
print("_età")
summary(positive$Age)
negative <- subset(diabetes,diabetes$Outcome == 0)
print("NEGATIVI: ")
print("_glucosio")
summary(negative$Glucose)
print("_età")
summary(negative$Age)



In [ ]:
ggplot(diabetes,aes(x=Glucose,y=Age,size=Insulin,color=Outcome))+geom_jitter(alpha=0.6)+scale_color_gradient(low = 'red', high = 'blue')+labs(title="Age and Glucose level against Insulin")

In [ ]:
qqnorm(positive_diabetes$Glucose,col="red")
qqline(positive_diabetes$Glucose)
qqnorm(positive_diabetes$Age,col="red")
qqline(positive_diabetes$Age)


In [ ]:
shapiro.test(positive_diabetes$Glucose)
shapiro.test(diabetes$Glucose)

shapiro.test(positive_diabetes$Age)


In [ ]:
library(MASS)
par(mfrow = c( 1 , 1))
boxcox(lm(positive_diabetes$Glucose~1), lambda = seq(-1,3,0.1))
title("Glucosio")
#
boxcox(lm(positive_diabetes$Age~1), lambda = seq(-3,3,0.1))
title("Età")

In [ ]:
l_g = 1.17
shapiro.test(((positive_diabetes$Glucose)^(l_g)-1)/(l_g))
l_a = -0.4
shapiro.test(((positive_diabetes$Age)^(l_a)-1)/(l_a))


In [ ]:
par(mfrow = c( 2 , 1))

qqnorm(positive_diabetes$Glucose,col="red")
qqline(positive_diabetes$Glucose)

qqnorm(((positive_diabetes$Glucose)^(l_g)-1)/(l_g),col="red")
qqline(((positive_diabetes$Glucose)^(l_g)-1)/(l_g))

qqnorm(positive_diabetes$Age,col="red")
qqline(positive_diabetes$Age)

qqnorm(((positive_diabetes$Age)^(l_a)-1)/(l_a),col="red")
qqline(((positive_diabetes$Age)^(l_a)-1)/(l_a))

In [ ]:
# TEST DI Kolmogorov-Smirnov: 

ks.test(positive_diabetes$Glucose, pnorm, mean(positive_diabetes$Glucose), sd(positive_diabetes$Glucose))
ks.test(diabetes$Glucose, pnorm, mean(diabetes$Glucose), sd(diabetes$Glucose))

ks.test(positive_diabetes$Age, pnorm, mean(positive_diabetes$Age), sd(positive_diabetes$Age))
ks.test(positive_diabetes$Insulin, pnorm, mean(positive_diabetes$Insulin), sd(positive_diabetes$Insulin))

In [ ]:
CT <- diabetes$Glucose
US <- diabetes$Age

#positive_diabetes <- diabetes[diabetes$Outcome == 1, ]
TN <- nrow(diabetes[diabetes$Age<30 & diabetes$Glucose<150,]) # true negatives
TN
FN <- nrow(diabetes[diabetes$Age<30 & diabetes$Glucose>=150,]) # false negatives
FN
FP <- nrow(diabetes[diabetes$Age>=30 & diabetes$Glucose<150,]) # false positives
FP
TP <- nrow(diabetes[diabetes$Age>=30 & diabetes$Glucose>=150,]) # true positives
TP
confusion_matrix <- matrix(c(TP,FP,FN,TN),byrow=T,nrow = 2)
confusion_matrix

In [ ]:
library(Epi)


accuracy <- (TP+TN)/(TP+FN+FP+TN)
accuracy
sensitivity <- TP/(TP+FN)
sensitivity
specificity <- TN/(FP+TN)
specificity

ROC( test = CT,
     stat = diabetes$Outcome,
     plot = c( "ROC"))

ROC( test = US,
     stat = diabetes$Outcome,
     plot = c( "ROC"))



In [ ]:
#MANN-WHITNEY TEST

#dati relazionali quindi anche ordinali!!!
wilcox.test(positive_diabetes$Glucose,negative_diabetes$Glucose,conf.int=T)
wilcox.test(positive_diabetes$Age,negative_diabetes$Age,conf.int=T)


In [ ]:
F1 <- ecdf(positive_diabetes$Glucose)
F2 <- ecdf(negative_diabetes$Glucose)
par(mfrow = c(1,1))
plot(F1,main="ecdf")
plot(F2,add=T,lty=2)


ks.test(positive_diabetes$Glucose,negative_diabetes$Glucose )
ks.test(positive_diabetes$Age,negative_diabetes$Age )

In [ ]:
# dropping null values in coloumns

